# Read a sample .txt file, and perform preprocessing.

In [1]:
import numpy as np  
import pandas as pd 
import re           
from bs4 import BeautifulSoup 
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import re                      # Regular expressions
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords   
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\R-k-l\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\R-k-l\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\R-k-l\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Using TensorFlow backend.


In [2]:
with open('the_boy_who_cried_wolf.txt', 'r') as file:
    text = file.read().replace('\n', '')
    file.close()
text

'A Shepherd Boy tended his master\'s Sheep near a dark forest not far from the village. Soon he found life in the pasture very dull. All he could do to amuse himself was to talk to his dog or play on his shepherd\'s pipe. One day as he sat watching the Sheep and the quiet forest, and thinking what he would do should he see a Wolf, he thought of a plan to amuse himself. His Master had told him to call for help should a Wolf attack the flock, and the Villagers would drive it away. So now, though he had not seen anything that even looked like a Wolf, he ran toward the village shouting at the top of his voice, "Wolf! Wolf!" As he expected, the Villagers who heard the cry dropped their work and ran in great excitement to the pasture. But when they got there they found the Boy doubled up with laughter at the trick he had played on them. A few days later the Shepherd Boy again shouted, "Wolf! Wolf!" Again the Villagers ran to help him, only to be laughed at again. Then one evening as the sun 

### Perform the following tasks on the data:
- Convert everything to lowercase
- Remove HTML tags
- Contraction mapping
- Remove (‘s)
- Remove any text inside the parenthesis ( )
- Eliminate punctuations and special characters
- Remove stopwords
- Remove shortwords

In [3]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [4]:
stop_words = set(stopwords.words('english')) 
def text_cleaner(text):
    """
    text (str): a string of text that can be multiple sentences to be parsed into a token stream. 
    Returns: a cleaned string of text
    """
    newString = text.lower()
    newString = BeautifulSoup(newString, exclude_encodings="lxml").text # removes html/xml taggs
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"','', newString)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    tokens = [w for w in newString.split() if not w in stop_words]
    long_words=[]
    for i in tokens:
        if len(i)>=3:                  #removing short word
            long_words.append(i)   
    return (" ".join(long_words)).strip()


In [16]:
text = text_cleaner(text)
print(type(text))
text

<class 'str'>


'shepherd boy tended master sheep near dark forest far village soon found life pasture dull could amuse talk dog play shepherd pipe one day sat watching sheep quiet forest thinking would see wolf thought plan amuse master told call help wolf attack flock villagers would drive away though seen anything even looked like wolf ran toward village shouting top voice wolf wolf expected villagers heard cry dropped work ran great excitement pasture got found boy doubled laughter trick played days later shepherd boy shouted wolf wolf villagers ran help laughed one evening sun setting behind forest shadows creeping pasture wolf really spring underbrush fall upon sheep terror boy ran toward village shouting wolf wolf though villagers heard cry run help cannot fool said wolf killed great many boy sheep slipped away forest'

## Try with and without a lemmitizer, 

In [6]:
def lemmatization(text):
    """
    text: a df string input
    Returns: 
    """
    lemmatizer = WordNetLemmatizer()
    new = []
    lem_words = [lemmatizer.lemmatize(x) for x in (words[:][:])]
    new.append(lem_words)
    return new
paragraph =[]
paragraph.append(text)
df_paragraph = pd.DataFrame(paragraph)
df_paragraph

,0
0,shepherd boy tended master sheep near dark forest far village soon found life pasture dull could amuse talk dog play shepherd pipe one day sat watching sheep quiet forest thinking would see wolf t...


In [7]:
lemmatizer = WordNetLemmatizer()
lem_text = lemmatizer.lemmatize(text)
type(lem_text)
lem_text == text

True

In [8]:
s_token = sent_tokenize(text)
s_token

['shepherd boy tended master sheep near dark forest far village soon found life pasture dull could amuse talk dog play shepherd pipe one day sat watching sheep quiet forest thinking would see wolf thought plan amuse master told call help wolf attack flock villagers would drive away though seen anything even looked like wolf ran toward village shouting top voice wolf wolf expected villagers heard cry dropped work ran great excitement pasture got found boy doubled laughter trick played days later shepherd boy shouted wolf wolf villagers ran help laughed one evening sun setting behind forest shadows creeping pasture wolf really spring underbrush fall upon sheep terror boy ran toward village shouting wolf wolf though villagers heard cry run help cannot fool said wolf killed great many boy sheep slipped away forest']

### See output of text length, to justify length of summary. Use root of the number of words input for now. 

# Seq2Seq

In [10]:
import keras
MAX_LEN = 1000
def seq2seq_model_builder(HIDDEN_DIM=300):
    encoder_inputs = Input(shape=(MAX_LEN, ), dtype='int32',)
    print(encoder_inputs)
    encoder_embedding = embed_layer(encoder_inputs)
    encoder_LSTM = LSTM(HIDDEN_DIM, return_state=True)
    encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)

    decoder_inputs = Input(shape=(MAX_LEN, ), dtype='int32',)
    decoder_embedding = embed_layer(decoder_inputs)
    decoder_LSTM = LSTM(HIDDEN_DIM, return_state=True, return_sequences=True)
    decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state=[state_h, state_c])

    # dense layer = Dense(VOCAB SIZE, activation='softmax')
    outputs = TimeDistributed(Dense(VOCAB_SIZE, activation='softmax'))(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], outputs)

    return model
# model = seq2seq_model_builder(HIDDEN_DIM=300)
# model.summary()

In [11]:
encoder_inputs = Input(shape=(MAX_LEN, ), dtype='int32',)
HIDDEN_DIM=20 # dimensionality of output space. Should be roughly root of the input.
# encoder_embedding = embed_layer(encoder_inputs) # create a 3D tensor
encoder_LSTM = LSTM(HIDDEN_DIM, return_state=True)
# encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)

In [12]:
encoder_inputs = Input(shape=(MAX_LEN, ), dtype='int32',)
encoder_LSTM = LSTM(HIDDEN_DIM, return_state=True)
encoder_outputs, state_h, state_c = encoder_LSTM(encoder_inputs)

ValueError: Input 0 of layer lstm_1 is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: [None, 1000]

In [14]:
def embedding_matrix_creater(embedding_dimention, word_index):
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dimention))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
          # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

def embedding_layer_creater(VOCAB_SIZE, EMBEDDING_DIM, MAX_LEN, embedding_matrix):
     embedding_layer = Embedding(input_dim = VOCAB_SIZE,
         output_dim = EMBEDDING_DIM,
         input_length = MAX_LEN,
         weights = [embedding_matrix],
         trainable = False)
     return embedding_layer

## Using GloVe (Global Vectors for word representation) to create an embeddings matrix

In [15]:
import time
timer_start = time.time()
embeddings_index = {}
with open('glove/glove.6B.200d.txt', encoding='utf-8') as f: # local path
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

print('Found %s word vectors.' % len(embeddings_index))
print("Creating embeddings_index took", round(time.time() - timer_start, 1), "seconds.")

Found 400000 word vectors.
Creating embeddings_index took 46.7 seconds.


In [29]:
from keras.preprocessing.text import Tokenizer
VOCAB_SIZE = 1000 # rough estimate. Could find a way to add ~500 to VOCAB_SIZE for every 30s or so. 
tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(list(text))
article_sequences = tokenizer.texts_to_sequences(text)
art_word_index = tokenizer.word_index
len(art_word_index)

24

In [30]:
tokenizer

In [31]:
art_word_index_1500 = {}
counter = 0
for word in art_word_index.keys():
    if art_word_index[word] == 0:
        print("found 0!")
        break
    if art_word_index[word] > VOCAB_SIZE:
        continue # see if the VOCAB_SIZE matches with the 
    else:
        art_word_index_1500[word] = art_word_index[word]
        counter += 1
print(counter)
sum_word_index_1500 = {}
counter = 0
for word in sum_word_index.keys():
    if sum_word_index[word] == 0:
        print("found 0!")
        break
    if sum_word_index[word] > VOCAB_SIZE:
        continue
    else:
        sum_word_index_1500[word] = sum_word_index[word]
        counter += 1
print(counter)

24

In [32]:
# Hyperparameters
MAX_LEN = 1000
VOCAB_SIZE =15000
EMBEDDING_DIM = 200
HIDDEN_UNITS = 200
VOCAB_SIZE = VOCAB_SIZE + 1
LEARNING_RATE = 0.002
BATCH_SIZE = 32
EPOCHS = 5

In [39]:
art_embedding_matrix = embedding_matrix_creater(200, word_index=art_word_index_1500)
print(art_embedding_matrix.shape)
# sum_embedding_matrix = embedding_matrix_creater(200, word_index=sum_word_index_1500)
# print(sum_embedding_matrix.shape)
encoder_embedding_layer = Embedding(input_dim = art_embedding_matrix.shape[0], 
                                    output_dim = art_embedding_matrix.shape[1],
                                    input_length = MAX_LEN,
                                    weights = [art_embedding_matrix],
                                    trainable = False)
# decoder_embedding_layer = Embedding(input_dim = 15000, 
#                                     output_dim = 200,
#                                     input_length = MAX_LEN,
#                                     weights = [sum_embedding_matrix],
#                                     trainable = False)

(25, 200)


In [40]:
"""
Simple LSTM Encoder-Decoder-seq2seq
"""
# encoder
encoder_inputs = Input(shape=(MAX_LEN, ), dtype='int32',)
encoder_embedding = encoder_embedding_layer(encoder_inputs)
encoder_LSTM = LSTM(HIDDEN_UNITS)(encoder_embedding)
# # decoder
# decoder_inputs = Input(shape=(MAX_LEN, ))
# decoder_embedding = decoder_embedding_layer(decoder_inputs)
# decoder_LSTM = LSTM(200)(decoder_embedding)
# # merge
# merge_layer = concatenate([encoder_LSTM, decoder_LSTM])
# decoder_outputs = Dense(units=VOCAB_SIZE+1, activation="softmax")(merge_layer) # SUM_VOCAB_SIZE, sum_embedding_matrix.shape[1]

# model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# model.summary()

200